# Colorspace - Foreground in Different Screen Modes

Create png files based on the screen dump from the atari800 emulation of colourspace.

Download our modified version of the atari800 emulator. This will allow us to create dump files of screen memory at specified trace points while colourspace is running.

In [156]:
import os
! rm -rf atari800
! git clone https://github.com/mwenge/atari800.git
os.chdir('atari800')
! git checkout memwatch
! ./autogen.sh
! ./configure --enable-monitormemorywatch
! make -j5
os.chdir('..')


Cloning into 'atari800'...
remote: Enumerating objects: 17525, done.
remote: Counting objects: 100% (1486/1486), done.
remote: Compressing objects: 100% (474/474), done.
remote: Total 17525 (delta 1052), reused 1406 (delta 1009), pack-reused 16039
Receiving objects: 100% (17525/17525), 5.75 MiB | 3.36 MiB/s, done.
Resolving deltas: 100% (13495/13495), done.
Branch 'memwatch' set up to track remote branch 'memwatch' from 'origin'.
Switched to a new branch 'memwatch'
configure.ac:204: installing './compile'
configure.ac:73: installing './install-sh'
configure.ac:73: installing './missing'
src/Makefile.am: installing './depcomp'

Now you need to run the configure script.  The configure script may take the
"--target=TARGET" option and various "--enable-FEATURE" and "--with-PACKAGE"
options.

Run "./configure --help" to see all available options.
Run "./configure --help=short" just to see the Atari800 FEATURE options.
Run "./configure --target=help" without a parameter to see the valid targ

config.status: executing depfiles commands
-------------------------------------------------------
                 CONFIGURATION RESULTS:
-------------------------------------------------------
Host OS...............................: linux
Target ...............................: default

Interface for video...................: sdl
Using screenshots?....................: yes
    Supported screenshot formats......: pcx png
Using cycle exact?....................: yes
Using the very slow computer support?.: no
Using the crash menu?.................: yes
Using the paged attribute array?......: no
Using per opcode cycles update?.......: no
Using the buffered log?...............: no
Using Altirra BIOS ROM?...............: yes
Using the monitor assembler?..........: yes
Using code breakpoints and history?...: yes
Using user-defined breakpoints?.......: no
Using monitor hints?..................: yes
Using 6502 opcode profiling?..........: no
Using TRACE monitor command?..........: no
Using rea

Run the atari800 emulator and write the screen memory dumps to a file.

In [174]:
! ./atari800/src/atari800 -memwatchbpc 44BF -memwatchfile foreground_screen_mode.txt ../psychedelia/bin/colourspace.xex

Using Atari800 config file: /home/robert/.atari800.cfg
Created by Atari 800 Emulator, Version 4.1.0

Memory Watch file open
Keyboard mapped to emulated joystick 0
Requested resolution 336x240 is not available, using 640x480 instead.
Video Mode: 336x240x32 windowed without vsync


Read in the ram history from our dump file.

In [175]:
log_file = "foreground_screen_mode.txt"
input_file = open(log_file,'r')

flatten = lambda l: [e for sublist in l for e in sublist]

ram_history = []
lines = input_file.readlines()
for i in range(1, len(lines), 131):
    raw_ram = [l[6:126].split() for l in lines[i:i+128]]
    raw_ram = [int(v,16) for l in raw_ram for v in l][:1440]
    pixel_ram = [raw_ram[v:v+40] for v in range(0, len(raw_ram), 40)]
    color_registers = [int(i,16) for i in lines[i+129].split()[1:10]]
    ram_history += [(pixel_ram, color_registers)]

Write a series of png files one for each stage of the evolution of the pattern

In [176]:
import colorspace_colors as cc

from PIL import Image, ImageColor

ROW_LINES = 5
def paintChar(origin, pixels, byte, col_register):
    c, r = origin
    x_o, y_o = c*8, r*ROW_LINES
    left_nibble = byte >> 4 & 0x0F 
    right_nibble = byte & 0x0F
    left_color = cc.color_value_to_html[col_register[left_nibble]]
    right_color = cc.color_value_to_html[col_register[right_nibble]]
    for n in range(0,4):
        for y in range(0,ROW_LINES):
            pixels[x_o+n, y_o+y] = ImageColor.getrgb(left_color)
    for n in range(0,4):
        for y in range(0,ROW_LINES):
            pixels[x_o+n+4, y_o+y] = ImageColor.getrgb(right_color)

for j, (scr, col_register) in enumerate(ram_history[-100:]):
    cols=len(scr[0])
    rows=len(scr)
    bits=8
    img = Image.new( 'RGB', (cols*bits,rows*ROW_LINES), "black")
    pixels = img.load()
    for y, l in enumerate(scr):
        for x, byte in enumerate(l):
            paintChar((x,y), pixels, byte, col_register)
    img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
    img.save(f"colorspace_foreground/pixel_pattern_{j}.png")



ValueError: unknown color specifier: ''